<a href="https://colab.research.google.com/github/hieu1462000/person_reID/blob/main/Person_reID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Torchreid


In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

In [ ]:
# cd to your preferred directory and clone this repo
!git clone https://github.com/KaiyangZhou/deep-person-reid.git

# create environment
%cd deep-person-reid/
!conda create --name torchreid python=3.7
!conda activate torchreid

# install dependencies
# make sure `which python` and `which pip` point to the correct path
!pip install -r requirements.txt

# install torch and torchvision (select the proper cuda version to suit your machine)
!conda install pytorch torchvision cudatoolkit=9.0 -c pytorch

# install torchreid (don't need to re-build it if you modify the source code)
!python setup.py develop



In [ ]:
!pip install Cython

In [ ]:
import torchreid
torchreid.models.show_avai_models()

#Datasets



In [ ]:
import torchreid
torchreid.models.show_avai_models()

['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x8d', 'resnet50_fc512', 'se_resnet50', 'se_resnet50_fc512', 'se_resnet101', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'densenet121_fc512', 'inceptionresnetv2', 'inceptionv4', 'xception', 'resnet50_ibn_a', 'resnet50_ibn_b', 'nasnsetmobile', 'mobilenetv2_x1_0', 'mobilenetv2_x1_4', 'shufflenet', 'squeezenet1_0', 'squeezenet1_0_fc512', 'squeezenet1_1', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1_0', 'shufflenet_v2_x1_5', 'shufflenet_v2_x2_0', 'mudeep', 'resnet50mid', 'hacnn', 'pcb_p6', 'pcb_p4', 'mlfn', 'osnet_x1_0', 'osnet_x0_75', 'osnet_x0_5', 'osnet_x0_25', 'osnet_ibn_x1_0', 'osnet_ain_x1_0', 'osnet_ain_x0_75', 'osnet_ain_x0_5', 'osnet_ain_x0_25']


In [ ]:
# Construct data manager
datamanager = torchreid.data.ImageDataManager(
    root ='reid-data',
    sources = 'market1501',
    targets = 'market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms = ['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3470 KB/s, 42 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




In [ ]:
# Construct OSnet model
model = torchreid.models.build_model(
    name='osnet_ain_x1_0', #change model here
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

# Initialise optimiser and learning rate scheduler
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

Downloading...
From: https://drive.google.com/uc?id=1-CaioD9NaqbHK_kzSMW8VE4_3KcsRjEo
To: /root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 29.5MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
# Construct engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
)

#Osnet_x1_0


In [ ]:
# Construct OSnet model
model = torchreid.models.build_model(
    name='osnet_x1_0', #change model here
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

# Initialise optimiser and learning rate scheduler
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

In [ ]:
# Construct engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
)

In [ ]:
#Run model training and test
engine.run(
    save_dir='log/OSNet',
    dist_metric='euclidean',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.484 (0.829)	data 0.001 (0.309)	eta 1:51:32	loss 6.5955 (6.6270)	acc 0.0000 (0.0000)	lr 0.000300
epoch: [1/20][20/404]	time 0.486 (0.654)	data 0.001 (0.155)	eta 1:27:53	loss 6.4737 (6.6023)	acc 0.0000 (0.4688)	lr 0.000300
epoch: [1/20][30/404]	time 0.481 (0.606)	data 0.002 (0.104)	eta 1:21:16	loss 6.6435 (6.5769)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/20][40/404]	time 0.481 (0.575)	data 0.001 (0.078)	eta 1:17:03	loss 6.4858 (6.5772)	acc 3.1250 (0.7812)	lr 0.000300
epoch: [1/20][50/404]	time 0.482 (0.556)	data 0.000 (0.062)	eta 1:14:28	loss 6.5911 (6.5476)	acc 6.2500 (1.2500)	lr 0.000300
epoch: [1/20][60/404]	time 0.488 (0.544)	data 0.002 (0.052)	eta 1:12:45	loss 6.3908 (6.5039)	acc 0.0000 (1.4062)	lr 0.000300
epoch: [1/20][70/404]	time 0.482 (0.536)	data 0.001 (0.045)	eta 1:11:30	loss 6.4499 (6.4774)	acc 6.2500 (2.0089)	lr 0.000300
epoch: [1/20][80/404]	time 0.476 (0.529)	data 0.001 (0.039)	eta 1:10:30	loss 6.7856 (6.4394)	acc 0.0000 (2.4219)	lr 0.000300


#Osnet_ibn_x1_0

In [ ]:
# Construct OSnet model
model = torchreid.models.build_model(
    name='osnet_ibn_x1_0', #change model here
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

# Initialise optimiser and learning rate scheduler
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

In [ ]:
# Construct engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
)

In [ ]:
engine.run(
    save_dir='log/OSNet',
    dist_metric='euclidean',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.484 (0.570)	data 0.002 (0.060)	eta 1:16:37	loss 6.6673 (6.6479)	acc 0.0000 (0.3125)	lr 0.000300
epoch: [1/20][20/404]	time 0.488 (0.529)	data 0.001 (0.031)	eta 1:11:04	loss 6.5956 (6.6229)	acc 0.0000 (0.6250)	lr 0.000300
epoch: [1/20][30/404]	time 0.506 (0.517)	data 0.001 (0.021)	eta 1:09:19	loss 6.5013 (6.6141)	acc 0.0000 (0.7292)	lr 0.000300
epoch: [1/20][40/404]	time 0.483 (0.510)	data 0.001 (0.016)	eta 1:08:19	loss 6.8070 (6.6197)	acc 0.0000 (0.8594)	lr 0.000300
epoch: [1/20][50/404]	time 0.495 (0.506)	data 0.000 (0.013)	eta 1:07:41	loss 6.4310 (6.5902)	acc 0.0000 (0.9375)	lr 0.000300
epoch: [1/20][60/404]	time 0.490 (0.504)	data 0.000 (0.011)	eta 1:07:23	loss 6.4267 (6.5633)	acc 3.1250 (1.3021)	lr 0.000300
epoch: [1/20][70/404]	time 0.496 (0.502)	data 0.000 (0.009)	eta 1:07:03	loss 6.0359 (6.5276)	acc 0.0000 (1.9643)	lr 0.000300
epoch: [1/20][80/404]	time 0.493 (0.501)	data 0.001 (0.008)	eta 1:06:46	loss 6.4160 (6.4967)	acc 6.2500 (2.5000)	lr 0.000300


#Osnet_ain_x1_0

In [ ]:
# Construct OSnet model
model = torchreid.models.build_model(
    name='osnet_ain_x1_0', #change model here
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

model = model.cuda()

# Initialise optimiser and learning rate scheduler
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)

In [ ]:
# Construct engine
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
)

In [ ]:
#Run model training and test
engine.run(
    save_dir='log/OSNet',
    dist_metric='cosine',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=False,
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/20][10/404]	time 0.515 (0.849)	data 0.001 (0.301)	eta 1:54:11	loss 6.5941 (6.6200)	acc 0.0000 (0.3125)	lr 0.000300
epoch: [1/20][20/404]	time 0.508 (0.681)	data 0.000 (0.151)	eta 1:31:29	loss 6.5248 (6.6100)	acc 0.0000 (0.1562)	lr 0.000300
epoch: [1/20][30/404]	time 0.512 (0.626)	data 0.000 (0.101)	eta 1:23:58	loss 6.5825 (6.5961)	acc 0.0000 (0.4167)	lr 0.000300
epoch: [1/20][40/404]	time 0.511 (0.598)	data 0.000 (0.076)	eta 1:20:08	loss 6.4385 (6.5766)	acc 0.0000 (0.5469)	lr 0.000300
epoch: [1/20][50/404]	time 0.515 (0.581)	data 0.000 (0.061)	eta 1:17:44	loss 6.2013 (6.5558)	acc 6.2500 (0.8125)	lr 0.000300
epoch: [1/20][60/404]	time 0.507 (0.570)	data 0.002 (0.051)	eta 1:16:09	loss 6.5858 (6.5231)	acc 3.1250 (1.1979)	lr 0.000300
epoch: [1/20][70/404]	time 0.515 (0.561)	data 0.001 (0.044)	eta 1:14:57	loss 6.0674 (6.4817)	acc 0.0000 (1.7411)	lr 0.000300
epoch: [1/20][80/404]	time 0.507 (0.555)	data 0.002 (0.039)	eta 1:14:01	loss 6.0390 (6.4435)	acc 6.2500 (2.1094)	lr 0.000300


#Visualize ranking results

In [ ]:
#Run model training and test
engine.run(
    save_dir='log/OSNet',
    max_epoch=20,
    eval_freq=10,
    print_freq=10,
    test_only=True,
    visrank=True,
)

##### Evaluating market1501 (source) #####
Extracting features from query set ...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Done, obtained 3368-by-512 matrix
Extracting features from gallery set ...
Done, obtained 15913-by-512 matrix
Speed: 0.0491 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...
** Results **
mAP: 70.4%
CMC curve
Rank-1  : 88.7%
Rank-5  : 95.9%
Rank-10 : 97.4%
Rank-20 : 98.5%
# query: 3368
# gallery 15913
Visualizing top-10 ranks ...
- done 100/3368
- done 200/3368
- done 300/3368
- done 400/3368
- done 500/3368
- done 600/3368
- done 700/3368
- done 800/3368
- done 900/3368
- done 1000/3368
- done 1100/3368
- done 1200/3368
- done 1300/3368
- done 1400/3368
- done 1500/3368
- done 1600/3368
- done 1700/3368
- done 1800/3368
- done 1900/3368
- done 2000/3368
- done 2100/3368
- done 2200/3368
- done 2300/3368
- done 2400/3368
- done 2500/3368
- done 2600/3368
- done 2700/3368
- done 2800/3368
- done 2900/3368
- done 3000/3368
- done 3100/3368
- done 3200/3368
- done 3300/3368
Done. Images have been saved to "log/OSNet/visrank_market1501" ...
